<div style="position: absolute; top: 0; left: 0; font-family: 'Garamond'; font-size: 14px;">
    <a href="https://github.com/patriciaapenat" style="text-decoration: none; color: inherit;">Patricia Peña Torres</a>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 48px;">
    <strong>Proyecto final, BRFSS-clustering</strong>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 36px;">
    <strong>2. Filtrado inicial</strong>
</div>

__________________

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>Dado que trabajamos con una base de datos extensa, es crucial examinar detenidamente la documentación. En este caso, me basé en las fuentes oficiales de la BBDD para explorar los datos. Comencé revisando el cuestionario (<a href="https://www.cdc.gov/brfss/questionnaires/pdf-ques/2022-BRFSS-Questionnaire-508.pdf" target="_blank">disponible aquí</a>) pero hay mayor concordancia con el codebook (<a href="https://www.cdc.gov/brfss/annual_data/2022/zip/codebook22_llcp-v2-508.zip" target="_blank">disponible aquí</a>), donde se encuentran los códigos asociados a las preguntas y respuestas. Esta revisión es esencial para comprender las preguntas formuladas y facilita la eliminación de secciones no pertinentes.
    En el presente notebook, llevé a cabo una revisión del documento mencionado y se realizó un primer filtrado con la finalidad de reducir el dataset con una limpieza rápida eliminando encuestas que no se habían completado o columnas que no utilizaremos.</normal>
</div>

<div style="font-family: 'Garamond'; font-size: 24px;">
    <strong>Importación de paquetes</strong>
</div>

In [1]:
import pandas as pd
import findspark
findspark.init()
import pyspark
import random
import os.path
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from pyspark.sql import DataFrame

<div style="font-family: 'Garamond'; font-size: 24px;">
    <strong>Configuración de Spark</strong>
</div>

In [2]:
# Si hay un SparkContext existente, debemos cerrarlo antes de crear uno nuevo
if 'sc' in locals() and sc:
    sc.stop()  # Detener el SparkContext anterior si existe

# Configuración de Spark
conf = (
    SparkConf()
    .setAppName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .setMaster("local[2]")  # Modo local con un hilo para ejecución
    .set("spark.driver.host", "127.0.0.1")  # Dirección del host del driver
    .set("spark.executor.heartbeatInterval", "3600s")  # Intervalo de latido del executor
    .set("spark.network.timeout", "7200s")  # Tiempo de espera de la red
    .set("spark.executor.memory", "14g")  # Memoria asignada para cada executor
    .set("spark.driver.memory", "14g")  # Memoria asignada para el driver
)

# Crear un nuevo SparkContext con la configuración especificada
sc = SparkContext(conf=conf)

# Configuración de SparkSession (interfaz de alto nivel para trabajar con datos estructurados en Spark)
spark = (
    SparkSession.builder
    .appName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .config("spark.sql.repl.eagerEval.enabled", True)  # Habilitar la evaluación perezosa en Spark SQL REPL
    .config("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Número máximo de filas a mostrar en la evaluación perezosa
    .getOrCreate()  # Obtener la sesión Spark existente o crear una nueva si no existe
) 

<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Lectura del archivo</strong>
</div>

In [3]:
# Ruta del archivo CSV
origin_path = r"C:\Users\patri\OneDrive - UAB\Documentos\GitHub\BRFSS-clustering\raw_data\BRFSS_2022.csv"

# Lee el archivo CSV y crea un DataFrame (especificando el separador "|")
df = spark.read.csv(origin_path, header=True, inferSchema=True)

In [4]:
# Abre el archivo en modo escritura y guarda los nombres de las columnas sin saltos de línea
with open(r"C:\Users\patri\Downloads\columnas_dataframe.txt", "w") as archivo:
    archivo.write(", ".join(map(str, df.columns)))

In [5]:
# Imprimir las dimensiones del DataFrame
print(f"Dimensiones del DataFrame: Filas={df.count()}, Columnas={len(df.columns)}")

Dimensiones del DataFrame: Filas=445132, Columnas=328


<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Trabajamos con un dataset muy extenso así que por ello lo reduciremos en este notebook.</normal>
</div>

<div style="font-family: 'Garamond'; font-size: 20px;">
    <strong>Limpieza</strong>
</div>

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Eliminamos las encuestas incompletas, aquellas en las que no se hablaba con una adulto o donde se referirá que no es buen momento para hablar</normal>
</div>

In [8]:
# Filtrar las filas basándose en si era un buen momento para hablar y si se hablaba con un mayor de edad
df = df.filter((col('SAFETIME') != 1) | (col('CADULT1') != 2) | (col('DISPCODE') == 1200) | (col('LADULT1') == 2)) 

In [9]:
# Imprimir las dimensiones del DataFrame
print(f"Dimensiones del DataFrame: Filas={df.count()}, Columnas={len(df.columns)}")

Dimensiones del DataFrame: Filas=363130, Columnas=328


<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Y ahora eliminamos varias consultas, entre ellas algunas identificativas, calculadas, planificación familiar o en general que no utilizaremos</normal>
</div>

In [10]:
# Columnas a eliminar
columns_to_drop = [
    "IDATE", "IMONTH", "IDAY", "IYEAR", "DISPCODE", "SEQNO", "_PSU", "CTELENM1", "PVTRESD1", "COLGHOUS", "STATERE1", 
    "CELPHON1", "LADULT1", "COLGSEX1", "LANDSEX1", "RESPSLCT", "SAFETIME", "CTELNUM1", "CELLFON5", "CADULT1", "CELLSEX1", 
    "PVTRESD3", "CCLGHOUS", "CSTATE1", "LANDLINE", "HHADULT", "SEXVAR", "NUMHHOL4", "NUMPHON4", "CPDEMO1C", "WEIGHT2", 
    "HEIGHT3","CADULT1","COVIDFS1","COVIDSE1", 'QSTVER', 'QSTLANG', '_METSTAT', '_URBSTAT', 'MSCODE', '_STSTR', 
    '_STRWT', '_RAWRAKE', '_WT2RAKE', '_IMPRACE', '_CHISPNC', '_CRACE2', '_CPRACE2', 'CAGEG', '_CLLCPWT', '_DUALUSE', '_DUALCOR', 
    '_LLCPWT2', '_LLCPWT', '_RFHLTH','_PHYS14D', '_MENT14D', '_HLTHPLN', '_HCVU652', '_TOTINDA', '_EXTETH3', '_ALTETH3', '_DENVST3', 
    '_MICHD', '_LTASTH1','_CASTHM1', '_ASTHMS1', '_DRDXAR2', '_PRACE2', '_MRACE2', '_HISPANC', '_RACE1', '_RACEG22', '_RACEGR4', 
    '_RACEPR1', '_AGEG5YR', '_AGE65YR', '_AGE_G', 'HTIN4', 'HTM4', 'WTKG3', '_BMI5CAT', '_RFBMI5', '_CHLDCNT', '_EDUCAG', '_INCOMG1',
    '_RFMAM22', '_MAM5023', '_HADCOLN', '_CLNSCP1', '_HADSIGM', '_SGMSCP1', '_SGMS101', '_RFBLDS5', '_STOLDN1', '_VIRCOL1',
    '_SBONTI1', '_CRCREC2', '_SMOKER3', '_RFSMOK3', '_CURECI2', '_YRSSMOK', '_PACKDAY', '_PACKYRS', '_YRSQUIT', '_SMOKGRP',
    '_LCSREC', 'DRNKANY6', 'DROCDY4_', '_RFBING6', '_DRNKWK2', '_RFDRHV8', '_FLSHOT7', '_PNEUMO3', '_AIDTST4','TOLDCFS','HAVECFS',
    'WORKCFS','FLSHTMY3','HIVTSTD3',  'RCSGEND1', 'RCSXBRTH', 'RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX', 'COVIDVA1', 'COVACGET', 'COVIDNU1'
    'COVIDINT','HADSEX','PFPPRVN4', ' TYPCNTR9', 'BRTHCNT4', 'WHEREGET', 'NOBCUSE8', 'BCPREFER'
]

# Eliminar
df_sin_columnas = df.drop(*columns_to_drop)

In [11]:
# Imprimir las dimensiones del DataFrame
print(f"Dimensiones del DataFrame: Filas={df_sin_columnas.count()}, Columnas={len(df_sin_columnas.columns)}")

Dimensiones del DataFrame: Filas=363130, Columnas=195


<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Exportación</strong>
</div>

<div style="font-family: 'Garamond'; font-size: 14px;">
    <normal>
Y por último exportamos</normal>
</div>

In [13]:
# Guardar el DataFrame como CSV
df_sin_columnas.write.csv("C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_Cleaner_2022.csv", header=True, mode="overwrite")